In [13]:
import sqlite3 
import pandas as pd
conn = sqlite3.connect('pets_database.db')
cur = conn.cursor()

In [5]:
cur.execute("""SELECT * from cats;""").fetchall()

[(1, 'Maru', 3, 'Scottish Fold', 1),
 (2, 'Hana', 1, 'Tabby', 1),
 (3, "Lil' Bub", 5, 'American Shorthair', None),
 (4, 'Moe', 10, 'Tabby', None),
 (5, 'Patches', 2, 'Calico', None),
 (6, None, None, 'Tabby', None)]

In [6]:
[x[0] for x in cur.description]

['id', 'name', 'age', 'breed', 'owner_id']

In [7]:
cur.execute("""SELECT * FROM cats ORDER BY age;""").fetchall()  # ASC|DESC

[(6, None, None, 'Tabby', None),
 (2, 'Hana', 1, 'Tabby', 1),
 (5, 'Patches', 2, 'Calico', None),
 (1, 'Maru', 3, 'Scottish Fold', 1),
 (3, "Lil' Bub", 5, 'American Shorthair', None),
 (4, 'Moe', 10, 'Tabby', None)]

In [8]:
cur.execute("""SELECT * FROM cats ORDER BY age DESC;""").fetchall() 

[(4, 'Moe', 10, 'Tabby', None),
 (3, "Lil' Bub", 5, 'American Shorthair', None),
 (1, 'Maru', 3, 'Scottish Fold', 1),
 (5, 'Patches', 2, 'Calico', None),
 (2, 'Hana', 1, 'Tabby', 1),
 (6, None, None, 'Tabby', None)]

In [13]:
cur.execute("""SELECT * FROM cats ORDER BY age DESC LIMIT 1;""").fetchall() 

[(4, 'Moe', 10, 'Tabby', None)]

In [11]:
cur.execute(
    """SELECT * 
    FROM cats 
    ORDER BY age DESC
    ;""").fetchall() 

[(4, 'Moe', 10, 'Tabby', None),
 (3, "Lil' Bub", 5, 'American Shorthair', None),
 (1, 'Maru', 3, 'Scottish Fold', 1),
 (5, 'Patches', 2, 'Calico', None),
 (2, 'Hana', 1, 'Tabby', 1),
 (6, None, None, 'Tabby', None)]

In [10]:
cur.execute(
    """SELECT name
    FROM cats 
    WHERE age BETWEEN 1 and 3
    ;""").fetchall() 

[('Maru',), ('Hana',), ('Patches',)]

In [9]:
cur.execute(
    """SELECT * 
    FROM cats
    WHERE name IS NULL
    ;""").fetchall() 

[(6, None, None, 'Tabby', None)]

In [8]:
cur.execute(
    """SELECT 
    COUNT(owner_id)
    FROM cats
    WHERE owner_id = 1
    ;""").fetchall() 

[(2,)]

### `GROUP BY`

Lastly, we'll talk about the handy aggregate function `GROUP BY`. Like its name
suggests, it groups your results by a given column.

Let's take our table of cats

```bash
id          name        age         breed          owner_id  
----------  ----------  ----------  -------------  ----------
1           Maru        3           Scottish Fold  1         
2           Hana        1           Tabby          1         
3           Lil\' Bub   5           American Shor            
4           Moe         10          Tabby                    
5           Patches     2           Calico                   
6                                   Tabby                    
```

Here, you can see at a glance that there are three tabby cats and
one of every other breed — but what if you had a larger database
where you couldn't just tally up the number of cats *grouped by*
breed? That's where — you guessed it! — `GROUP BY` comes in handy.

``` sql
SELECT breed, COUNT(breed) FROM cats GROUP BY breed;
```

In [15]:
cur.execute("""
                SELECT breed 
                FROM cats 
                GROUP BY breed
                ;""") 

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,breed
0,American Shorthair
1,Calico
2,Scottish Fold
3,Tabby


In [23]:
cur.execute(
    """
    SELECT 
    breed
    ,owner_id
    ,COUNT(breed) 
    FROM cats 
    GROUP BY breed, owner_id
    ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,breed,owner_id,COUNT(breed)
0,American Shorthair,NaN,1
1,Calico,NaN,1
2,Scottish Fold,1.0,1
3,Tabby,NaN,2
4,Tabby,1.0,1


In [24]:
cur.execute("""SELECT cats.name FROM cats;""").fetchall() 

[('Maru',), ('Hana',), ("Lil' Bub",), ('Moe',), ('Patches',), (None,)]

In [25]:
conn = sqlite3.connect('dogs.db')
cur = conn.cursor()

In [28]:
cur.execute(
    """
    SELECT 
    name
    ,breed
    FROM dogs
    ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,name,breed
0,Snoopy,beagle
1,McGruff,bloodhound
2,Scooby,great dane
3,Little Ann,coonhound
4,Pickles,black lab


In [30]:
cur.execute(
    """
    SELECT name 
    FROM dogs
    ORDER BY name
    ;""")  # ASC|DES

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,name
0,None
1,Clifford
2,Lassie
3,Little Ann
4,McGruff


In [32]:
cur.execute("""
                SELECT name, breed
                FROM dogs
                WHERE  hungry = 1
                ORDER BY age
                
            ;""").fetchall()

[('Snoopy', 'beagle'),
 ('Clifford', 'big red'),
 (None, 'golden retriever'),
 ('Scooby', 'great dane'),
 ('Lassie', 'collie'),
 ('Pickles', 'black lab')]

In [33]:
cur.execute("""
                SELECT name, age, temperament
                FROM dogs
                ORDER BY age DESC
                LIMIT 1
                
            ;""").fetchall()

[('Pickles', 13, 'mischievous')]

In [34]:
cur.execute("""
                SELECT *
                FROM dogs
                ORDER BY age
                LIMIT 3
                
            ;""").fetchall()

[(1, 'Snoopy', 3, 'M', 'beagle', 'friendly', 1),
 (6, 'Clifford', 4, 'M', 'big red', 'smiley', 1),
 (9, None, 4, 'M', 'golden retriever', 'playful', 1)]

In [35]:
cur.execute("""
                SELECT name, breed
                FROM dogs
                WHERE age BETWEEN 5 and 10 
                ORDER BY age DESC
                
            ;""").fetchall()

[('McGruff', 'bloodhound'),
 ('Snowy', 'fox terrier'),
 ('Lassie', 'collie'),
 ('Scooby', 'great dane'),
 ('Little Ann', 'coonhound')]

### Select the name, age, and hungry columns for hungry dogs between the ages of two and seven.  This query should also list these dogs in alphabetical order.

In [59]:
import pandas as pd

In [36]:
df = pd.DataFrame(cur.execute(
    """
    SELECT name
    ,age
    ,hungry
    FROM dogs
    WHERE age BETWEEN 2 and 7 
    ORDER BY name
    ;""").fetchall())
df.columns = [x[0] for x in cur.description]
df

,name,age,hungry
0,None,4,1
1,Clifford,4,1
2,Lassie,7,1
3,Little Ann,5,0
4,Scooby,6,1
5,Snoopy,3,1
